Q3 QUADRATIC CHATGPT

In [1]:
import pandas as pd
from gurobipy import Model, GRB
import numpy as np

/var/folders/9w/090_7gb54d54ppm4sfqk8xgr0000gn/T/ipykernel_33530/3415466315.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [3]:
from gurobipy import Model, GRB

# Define the data
data = {
    'Product Line': ['Line 1', 'Line 1', 'Line 1', 'Line 2', 'Line 2', 'Line 2', 'Line 3', 'Line 3', 'Line 3'],
    'Version': ['Basic', 'Advanced', 'Premium', 'Basic', 'Advanced', 'Premium', 'Basic', 'Advanced', 'Premium'],
    'Intercept': [135234.55, 237790.24, 335675.33, 137041.38, 236846.14, 335827.02, 139414.27, 235991.95, 339313.32],
    'Sensitivity': [-45.8964, -8.22779, -7.58444, -9.03317, -4.42787, -2.62906, -2.42148, -4.00051, -2.29662],
    'Capacity': [80020.0, 89666.0, 80638.0, 86740.0, 84050.0, 86565.0, 87051.0, 85156.0, 87588.0]
}

# Create a DataFrame from the data
df = pd.DataFrame(data)

# Create the Gurobi model
m = Model('Quadratic Optimization')

# Define the price variables for each version of each product line
prices = {}
for idx, row in df.iterrows():
    prices[(row['Product Line'], row['Version'])] = m.addVar(lb=0, name=f"price_{row['Product Line']}_{row['Version']}")

# Define the demand functions for each version of each product line
def demand(line, version, p):
    intercept = df[(df['Product Line'] == line) & (df['Version'] == version)]['Intercept'].values[0]
    sensitivity = df[(df['Product Line'] == line) & (df['Version'] == version)]['Sensitivity'].values[0]
    return intercept - sensitivity * p

# Define the revenue function to be maximized
revenue = sum(prices[(line, version)] * demand(line, version, prices[(line, version)]) for line in df['Product Line'].unique() for version in ['Basic', 'Advanced', 'Premium'])

# Add constraints to ensure that the prices increase within each product line
for line in df['Product Line'].unique():
    m.addConstr(prices[(line, 'Basic')] <= prices[(line, 'Advanced')])
    m.addConstr(prices[(line, 'Advanced')] <= prices[(line, 'Premium')])

# Set the objective function to maximize revenue
m.setObjective(revenue, GRB.MAXIMIZE)

# Solve the model
m.optimize()

# Print the optimal revenue
print(f'Optimal revenue: ${m.objVal}')


Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[arm] - Darwin 23.3.0 23D60)

CPU model: Apple M2
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 6 rows, 9 columns and 12 nonzeros
Model fingerprint: 0x8a3abd80
Model has 9 quadratic objective terms
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+05, 3e+05]
  QObjective range [5e+00, 9e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [0e+00, 0e+00]

Continuous model is non-convex -- solving as a MIP

Found heuristic solution: objective -0.0000000
Found heuristic solution: objective 5.004250e+19
Presolve time: 0.00s
Presolved: 16 rows, 20 columns, 40 nonzeros
Presolved model has 9 bilinear constraint(s)
         in product terms.
         Presolve was not able to compute smaller bounds for these variables.
         Consider bounding these variables or reformulating the model.

Found heuristic solution: objective 3.581677e+29
Variable types:

______

Q3 REVISED

In [4]:
import pandas as pd
from gurobipy import Model, GRB

# Load the dataset
df = pd.read_csv('/Users/pratiksha/Downloads/price_response.csv')  

# Create a Gurobi model
model = Model('Maximize Revenue Within Line Constraints')

# Convert dataframe columns to arrays
intercepts = df['Intercept'].to_numpy()
sensitivities = abs(df['Sensitivity'].to_numpy())
capacities = df['Capacity'].to_numpy()

# Add price variables for each product
prices = model.addVars(len(df), name='price', lb=0)

# Define the objective function (total revenue)
revenue = sum((intercepts[i] - sensitivities[i] * prices[i]) * prices[i] for i in range(len(df)))
model.setObjective(revenue, GRB.MAXIMIZE)

# Add capacity constraints
for i in range(len(df)):
    demand = intercepts[i] - sensitivities[i] * prices[i]
    model.addConstr(demand <= capacities[i], name=f'capacity_{i}')


#Add constraints for price ordering within each product line
num_products_per_line = 3
for i in range(0, len(df), num_products_per_line):
    model.addConstr(prices[i+1] - prices[i] >= 0.01, name=f'price_order_{i}_basic_advanced')
    model.addConstr(prices[i+2] - prices[i+1] >= 0.01, name=f'price_order_{i}_advanced_premium')


# Optimize the model
model.optimize()

# Print the optimal prices and total revenue
if model.status == GRB.OPTIMAL:
    print("Optimal Prices:")
    for i in range(len(df)):
        print(f"{df.loc[i, 'Product']}: ${prices[i].X:.2f}")
    print(f"Total Revenue: ${model.ObjVal:.2f}")
else:
    print("No optimal solution found.")

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[arm] - Darwin 23.3.0 23D60)

CPU model: Apple M2
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 15 rows, 9 columns and 21 nonzeros
Model fingerprint: 0xc90e07c7
Model has 9 quadratic objective terms
Coefficient statistics:
  Matrix range     [1e+00, 5e+01]
  Objective range  [4e+04, 4e+04]
  QObjective range [5e+00, 9e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e-02, 5e+04]
Presolve removed 9 rows and 0 columns
Presolve time: 0.00s
Presolved: 6 rows, 9 columns, 12 nonzeros
Presolved model has 9 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 AA' NZ     : 3.000e+00
 Factor NZ  : 9.000e+00
 Factor Ops : 1.500e+01 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0   5.34101424e+08  1.18618541e+09  5.00e+02 2.21e+02  